In [2]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import cv2

def create_temporal_rgb_frame(df, ts_0, ts_1, height=200, width=200):
    """
    Create a temporal RGB frame from event data within a given time window.

    Args:
        df (pd.DataFrame): DataFrame containing event data with columns ['t', 'x', 'y', 'p'].
        ts_0 (float): Start timestamp (in microseconds).
        ts_1 (float): End timestamp (in microseconds).
        height (int): Height of the output image.
        width (int): Width of the output image.

    Returns:
        np.ndarray: An (height, width, 3) uint8 RGB image where each color channel encodes events
                    from a different third of the time window:
                        - Red: events in the first third,
                        - Green: events in the middle third,
                        - Blue: events in the last third.
    """
    ev_frame = np.zeros([height, width, 3], dtype=np.uint8)
    
    if df.empty or (ts_1 <= ts_0):
        return ev_frame

    window = ts_1 - ts_0
    w1 = ts_0 + window / 3
    w2 = ts_1 - window / 3

    # --- red (1/3) ---
    red_ev = df.loc[df['t'].between(ts_0, w1)]
    if not red_ev.empty:
        x = red_ev['x'].to_numpy()
        y = red_ev['y'].to_numpy()
        t = red_ev['t'].to_numpy()
        
        alpha = (t - ts_0) / window
        ev_frame[y, x, 0] = np.clip(alpha * 255.0, 0, 255)

    # --- green (2/3) ---
    green_ev = df.loc[df['t'].between(w1, w2)]
    if not green_ev.empty:
        x = green_ev['x'].to_numpy()
        y = green_ev['y'].to_numpy()
        t = green_ev['t'].to_numpy()

        alpha = (t - ts_0) / window
        ev_frame[y, x, 1] = np.clip(alpha * 255.0, 0, 255)

    # --- blue (3/3) ---
    blue_ev = df.loc[df['t'].between(w2, ts_1)]
    if not blue_ev.empty:
        x = blue_ev['x'].to_numpy()
        y = blue_ev['y'].to_numpy()
        t = blue_ev['t'].to_numpy()

        alpha = (t - ts_0) / window
        ev_frame[y, x, 2] = np.clip(alpha * 255.0, 0, 255)
        
    return ev_frame

def create_temporal_mono_frame(df, ts_0, ts_1, height=200, width=200):
    """
    Create a temporal mono frame from event data within a given time window.

    Args:
        df (pd.DataFrame): DataFrame containing event data with columns ['t', 'x', 'y', 'p'].
        ts_0 (float): Start timestamp (in microseconds).
        ts_1 (float): End timestamp (in microseconds).
        height (int): Height of the output image.
        width (int): Width of the output image.

    Returns:
        np.ndarray: An (height, width, 1) uint8 Mono image where one channel encodes events
    """
    ev_frame = np.zeros([height, width, 1], dtype=np.uint8)
    
    if df.empty or (ts_1 <= ts_0):
        return ev_frame

    x = df['x'].to_numpy()
    y = df['y'].to_numpy()
    t = df['t'].to_numpy()
        
    alpha = (t - ts_0) / (ts_1 - ts_0)
    ev_frame[y, x, 0] = (255.0 * np.clip(alpha*0.2 + 0.8, 0, 1) + 0.5).clip(0, 255.0).astype(np.uint8) 
    
    return ev_frame

def generate_event_images(filename, domain='train' , outdir='./dataset/train', divider=1, img_height=200, img_width=200, mode='mono'):
    """
    Generate and save temporal event images from event-based data.

    Args:
        filename (str): Name of the .npz file (without extension) containing event data.
        domain (str): Subdirectory under './data' and './dataset' (e.g., 'train', 'test').
        outdir (str): Output directory to save generated images.
        divider (int): Number of subdivisions per timestamp interval for interpolation.
        img_height (int): Height of the output images.
        img_width (int): Width of the output images.
    """
    datapath = './data'
    fpath = os.path.join(datapath, domain, filename + '.npz')
    outpath = os.path.join(outdir, filename + '_' + str(divider), 'img')
    os.makedirs(outpath, exist_ok=True)
    
    sequence = np.load(fpath)
    events = sequence['events']
    timestamps = sequence['timestamps'] 
    # traj = sequence['traj']
    # range_meter = sequence['range_meter']

    ev_data = pd.DataFrame(events, columns=['t', 'x', 'y', 'p']).set_index('t').sort_index()
    # ts_data = pd.DataFrame(t, columns=['time/s'])
    # traj_data = pd.DataFrame(x, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'roll', 'pitch', 'yaw','wr', 'wp', 'wy'])
    # rm_data = pd.DataFrame(rm, columns=['time/s', 'd/m'])

    new_ts_list = []
    if divider == 1:
        new_ts_list = timestamps
        ts_us = new_ts_list * 1e6
    else: 
        for i in range(len(timestamps) - 1):
            start_time = timestamps[i]
            end_time = timestamps[i+1]
            interpolated_times = np.linspace(start_time, end_time, divider, endpoint=False)
            new_ts_list.append(interpolated_times)
    
        new_ts_list.append(np.array([timestamps[-1]]))
        ts_us = np.concatenate(new_ts_list) * 1e6
        
    event_groups_list = []

    for i in tqdm(range(len(ts_us))):
        if (i != len(ts_us) - 1): 
            ts_0 = ts_us[i]
            ts_1 = ts_us[i+1]
            df = ev_data.loc[ts_0 : ts_1 - 1].reset_index()
        else:
            ts_0 = ts_us[i]
            ts_1 = 2 * ts_0 - ts_us[i - 1]
            df = ev_data.loc[ts_0 : ts_1 - 1].reset_index()

        df_pos = df[df['p'] == True] 
        df_neg = df[df['p'] == False]
        
        imgname_pos = os.path.join(outpath, f"{i:04d}_pos.png")
        imgname_neg = os.path.join(outpath, f"{i:04d}_neg.png")

        if mode == 'rgb':
            rgb_pos = create_temporal_rgb_frame(df_pos, ts_0, ts_1, img_height, img_width)
            bgr_pos = cv2.cvtColor(rgb_pos, cv2.COLOR_RGB2BGR)
            rgb_neg = create_temporal_rgb_frame(df_neg, ts_0, ts_1, img_height, img_width)
            bgr_neg = cv2.cvtColor(rgb_neg, cv2.COLOR_RGB2BGR)
            cv2.imwrite(imgname_pos, bgr_pos)
            cv2.imwrite(imgname_neg, bgr_neg)
        elif mode == 'mono':
            mono_pos = create_temporal_mono_frame(df_pos, ts_0, ts_1, img_height, img_width)
            mono_neg = create_temporal_mono_frame(df_neg, ts_0, ts_1, img_height, img_width)
            cv2.imwrite(imgname_pos, mono_pos)
            cv2.imwrite(imgname_neg, mono_neg)
        
    print(f"Save {len(ts_us)} couples of imgs(pos/neg) at {outpath}. ")

def generate_traj_data(filename, domain='train' , outdir='./dataset', divider=1):
    """
    根据输入的npz文件,产生所有时间窗口的轨迹的插值
    Args:
        filename (str): 读取的npz文件名 (不含扩展名)。
        domain (str): 'train' 或 'test' 等子目录。
        outdir (str): 输出数据的根文件夹。
        divider (int): 时间戳间隔的细分数量。
    """
    datapath = './data'
    fpath = os.path.join(datapath, domain, filename + '.npz')
    outpath = os.path.join(outdir, filename + '_' + str(divider), 'data')
    os.makedirs(outpath, exist_ok=True)
    
    sequence = np.load(fpath)
    traj = sequence['traj']
    timestamps = sequence['timestamps'] 
    range_meter = sequence['range_meter']

    traj_new_list = []
    rm_new_list = []

    if divider == 1:
        ts_new = timestamps
        traj_new = traj
    else: 
        ts_new_list = []
        for i in range(len(timestamps) - 1):
            start_time = timestamps[i]
            end_time = timestamps[i+1]
            interpolated_times = np.linspace(start_time, end_time, divider, endpoint=False)
            ts_new_list.append(interpolated_times)

            start_vec = traj[i] 
            end_vec = traj[i+1] 
            interpolated_vec = np.linspace(start_vec, end_vec, divider, endpoint=False)
            traj_new_list.append(interpolated_vec)
    
        ts_new_list.append(np.array([timestamps[-1]]))
        ts_new = np.concatenate(ts_new_list)
        traj_new_list.append(traj[-1].reshape(1, -1))
        traj_new = np.concatenate(traj_new_list, axis=0)

    range_orig_times = range_meter[:, 0]
    range_orig_dists = range_meter[:, 1]
    range_new_dists = np.interp(ts_new, range_orig_times, range_orig_dists)
    
    df = pd.DataFrame(traj_new[:,2:], columns=['z','vx', 'vy', 'vz', 'roll', 'pitch', 'yaw','wr', 'wp', 'wy'])
    df['rangemeter'] = range_new_dists
    csvpath = os.path.join(outpath, 'trajectory.csv')
    df.to_csv(csvpath, index=False)
    print(f"Save {len(df)} lines of trajectory data to {csvpath}.")

In [4]:
for i in [1]:
    for j in range(0, 28):
        file_num = str(j).zfill(4)
        # generate_event_images(file_num, domain='train' , outdir='./dataset/train', divider=i, mode='mono')
        generate_traj_data(file_num, domain='train' , outdir='./dataset/trainv2', divider=i)

Save 120 lines of trajectory data to ./dataset/trainv2/0000_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0001_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0002_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0003_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0004_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0005_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0006_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0007_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0008_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0009_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0010_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/trainv2/0011_1/data/trajectory.csv.
Save

In [ ]:
for i in [1]:
    for j in range(0, 28):
        file_num = str(j).zfill(4)
        generate_event_images(file_num, domain='train' , outdir='./dataset/pretrainv2', divider=i, mode='mono')

100%|██████████| 120/120 [00:03<00:00, 36.62it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/pretrainv2/0000_1/img. 


100%|██████████| 120/120 [00:02<00:00, 49.92it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/pretrainv2/0001_1/img. 


100%|██████████| 120/120 [00:00<00:00, 151.18it/s]


...

Save 120 couples of imgs(pos/neg) at ./dataset/pretrainv2/0025_1/img. 


100%|██████████| 120/120 [00:00<00:00, 426.50it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/pretrainv2/0026_1/img. 


100%|██████████| 120/120 [00:01<00:00, 89.10it/s]

Save 120 couples of imgs(pos/neg) at ./dataset/pretrainv2/0027_1/img. 


In [3]:
for i in [1]:
    for j in range(28, 93):
        file_num = str(j).zfill(4)
        # generate_event_images(file_num, domain="test", outdir='./dataset/testv2', divider=i, mode='mono')
        generate_traj_data(file_num, domain="test", outdir='./dataset/testv2', divider=i)

Save 120 lines of trajectory data to ./dataset/testv2/0028_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0029_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0030_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0031_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0032_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0033_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0034_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0035_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0036_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0037_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0038_1/data/trajectory.csv.
Save 120 lines of trajectory data to ./dataset/testv2/0039_1/data/trajectory.csv.
Save 120 lines o